# Explore feature-to-feature relationship in Titanic

In [1]:
import pandas as pd
from sklearn import datasets
import discover

In [2]:
df = pd.read_csv("titanic_train.csv")
df['SibSpParch'] = df.SibSp + df.Parch

cols = ['Pclass', 'SibSp', 'Parch', 'SibSpParch', 'Age', 'Fare', 'Sex', 'Embarked', 'Survived']
classifier_overrides = set(['Embarked', 'Survived', 'Sex'])

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SibSpParch
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df_results = discover.discover(cols, classifier_overrides, df)

In [5]:
df_results.pivot(index='target', columns='feature', values='score').fillna(1) \
.style.background_gradient(cmap="viridis", low=0.7, axis=1) \
.set_precision(2)

feature,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,SibSpParch,Survived
target,,,,,,,,,
Age,1,-0.027,-0.079,0.065,0.18,0.0045,0.13,0.066,0.00055
Embarked,0.79,1,0.91,0.73,0.73,0.73,0.73,0.73,0.73
Fare,-0.26,0.082,1,0.027,0.35,0.042,0.03,0.036,0.077
Parch,-0.17,0.005,0.29,1,-0.0023,0.071,0.15,0.62,0.011
Pclass,0.073,0.1,0.89,0.00054,1,-0.0067,0.026,0.027,0.1
Sex,0.53,0.64,0.69,0.66,0.63,1,0.64,0.66,0.78
SibSp,-0.06,-0.0094,0.55,0.28,0.00024,0.012,1,0.78,-0.004
SibSpParch,0.054,-0.00072,0.71,0.68,0.0012,0.045,0.78,1,-0.0025
Survived,0.53,0.62,0.73,0.64,0.68,0.78,0.62,0.66,1
